In [1]:
import import_ipynb #Allows you to import ipynb files (TEMPORARY?)
from AStar import AStar

importing Jupyter notebook from AStar.ipynb


In [2]:
#Testing AStar

maze = [[0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]]
graph = [[0, 1, 0, 0, 0, 0],
         [1, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 1],
         [0, 0, 0, 0, 1, 0],
         [0, 1, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0]]
start = (0, 0)
end = (5, 5)
pf = AStar(graph, start, end)
path = pf.run()
print(path)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]


In [ ]:
#Testing Hybrid AStar

In [ ]:
#Overall Algorithm
#First the car will report any obstacles in its way
#if any, add to map w/ appropriate location
#then perform a* algorithm with global map
#do a check if there's a collision
#if so, run machine learning algorithm to change, unless there are no changes to the map
#Should only use machine learning algo IF and only IF there are changes to the map
#return path to car, use route instructions to move car appropriately

In [ ]:
#To-Do:
#figure out how to optimize this for cars (take steering to account)
#figure out how to make initial node fat
#if there is a better algorithm, since A* is memory intensive, do it

In [ ]:
#Algo notes:
#Start/goal node = Tuple (x,y,theta) x and y being the world position of the rear-axis of the car, and
#theta being the angle of the car according to the z axis

#returns a series of nodes of s0...sg to have an optimal path from s0 to sg, then the trajectory is
#smoothed using gradient descent

#using A* using the four dimensional node configeration (x, y, theta, forward/backward) with 6 dir.
#in mind with a certain distance

#distance heuristic



#grid notes:
#voronei fields


In [ ]:
#he generate map to calculate distances from one cell to all cells with all cells
#Reeds sheppe used to calculated the heuristic in the A* algo
#A* = F(X) = g(x) + h(x)

#ndoe i = current
#node j = next
#g(j) = g(i) + dist(ij)(1+v*voro(i)+p*rev(j))+delta*switch(j)

#Dist(ij) = eculidean distance between nodes i and j
#rev(j) = direction of motion (0, 1)
#   rev discourages backwards travel

#voro(j) = value of the voronoi field at node j
#  voro pushes pathing away from obstacles

#switch(ij) = 1 if i and j directions are diff, 0 otherwise
# penalizes switching directions

#v, p, and deleta are cosntant penalty factors

#page 30 of the paper we're using
